# Завантаження тестових даних

## Допоміжна документація
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Підключення до БД через "тонкий" клієнт ORACLE та виборка одного запису з його виводом через "print"
- Очікується, що вибиирається тільки один запис [5.1.1. Fetch Methods](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)
- Прив'язка зміних запиту виконується через **змінні прив'язки**

In [14]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

Enter password:  ········


## Підготовка тестових даних

In [15]:
import oracledb
import os
from faker import Faker

def next_custid(numcust):
    """Генерація значення CUSTID. numcust - ціле число"""
    # Задана фіксована довжина
    FIXED_LENGTH = 9
    # 1. Спочатку перетворюємо int на str
    number_str = str(numcust) 
    # 2. Використовуємо zfill() для додавання ведучих нулів
    result_zfill = number_str.zfill(FIXED_LENGTH)
    return result_zfill

def insert_to_doc( CUSTID, TYPEDOC, ISACRUAL):
    """INSERT CUSTOMER DOCUMET TYPE"""
    try:
        sql_insert_doc = """ 
            INSERT INTO CUSTDOC.CUST$DOCS
            (CUSTID, TYPEDOC, ISACRUAL )
            VALUES
            (:CUSTID, :TYPEDOC, :ISACRUAL)
             RETURNING IDDOC INTO :returned_iddoc 
        """       
        returned_iddoc = cursor.var(oracledb.NUMBER)
        cursor.execute(sql_insert_doc, {
            'CUSTID': CUSTID,
            'TYPEDOC': TYPEDOC,
            'ISACRUAL': ISACRUAL, 
            'returned_iddoc': returned_iddoc
        })

        return returned_iddoc.getvalue()[0]
    except oracledb.Error as e:
        # Обробка помилок Oracle
        error_obj, = e.args
        print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
    except Exception as error:
        print(f" Виникла неочікувана помилка: {type(error).__name__}")
        print(f"Деталі помилки: {error}")
        print(sys.exc_info())          


def insert_to_blob( custid, iddoc, file_path,  file_desc):
    """Прочитати файл та завантажити його в BLOB поле таблиці ORACLE"""
    try:
        with open(file_path, 'rb') as f:
                file_data = f.read()
    
        file_name = os.path.basename(file_path)
        sql_insert_blob = """
            INSERT INTO CUSTDOC.CUST$DOCS$ATTACH 
            (IDDOC, FILE_NAME, FILE_DESC, FILE_CONTENT)
            VALUES 
            (:returned_iddoc, :file_name,  :file_desc, :file_content)
            RETURNING IDFL INTO :returned_idfl """
        returned_idfl = cursor.var(oracledb.NUMBER)
        cursor.execute(sql_insert_blob, {
            'returned_iddoc': iddoc,
            'file_name': 'CU_' + custid + '_' + file_name,
            'file_desc': file_desc,
            'file_content': file_data, 
            'returned_idfl': returned_idfl
        })
        return returned_idfl.getvalue()[0]
    except oracledb.Error as e:
        # Обробка помилок Oracle
        error_obj, = e.args
        print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
    except Exception as error:
        print(f" Виникла неочікувана помилка: {type(error).__name__}")
        print(f"Деталі помилки: {error}")
        print(sys.exc_info())          
            
#=============================================================
try:
    # Підключаюся до БД
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$CUST"""
    cursor.execute(sqld) 
    connection.commit();
    #if cursor:
    #    cursor.close()
    
    
    sql = """insert into CUSTDOC.CUST$CUST 
              ( CUSTID, CUSTTYPE, CUSTNAME ) VALUES 
              ( :A_CUSTID, :A_CUSTTYPE, :A_CUSTNAME )"""

    custid_int=0
    fake_en = Faker('en_US') 
    print("--- Фізичні особи (Англійська) ---")
    for _ in range(56):
        # Генерація повного імені
        full_name = fake_en.name() 
       
        custid_int+=1
        custid=next_custid(custid_int) 
        #print(f"{full_name} for custi={custid}")
        cursor.execute(sql, 
                       {
                        "A_CUSTID": custid, 
                        "A_CUSTTYPE": "CI", 
                        "A_CUSTNAME": full_name 
                        }
         )

        if custid_int % 2 == 0:
            ret_iddoc=insert_to_doc(custid, 'FOR_PASS', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document: IMG_20190512_123820.jpg')  
        elif  custid_int % 3 == 0:
            ret_iddoc=insert_to_doc(custid, 'NAT_ID', 'Y')
            
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document:  IMG_20190512_123820.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124107.jpg',  'Document:  IMG_20190512_124107.jpg')    
            
            ret_iddoc=insert_to_doc(custid, 'TAX_ID', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190515_135249.jpg',  'Document:  IMG_20190515_135249.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124206.jpg',  'Document:  IMG_20190512_124206.jpg')    
        elif  custid_int % 5 == 0:
            ret_iddoc=insert_to_doc(custid, 'DRV_LIC', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001465.jpg',  '1000001465.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001462.jpg',  '1000001462.jpg') 
        else:    
            ret_iddoc=insert_to_doc(custid, 'NAT_ID', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190511_152919.jpg',  'Document:   IMG_20190511_152919.jpg')    
    connection.commit()
    print("--- Legal Ent особи (Англійська) ---")
    for _ in range(56):
        # Генерація company name
        company_name = fake_en.company()
        custid_int+=1
        custid=next_custid(custid_int) 
        #print(f"{company_name} for custi={custid}")
        cursor.execute(sql, 
                       {
                        "A_CUSTID": custid, 
                        "A_CUSTTYPE": "LE", 
                        "A_CUSTNAME": company_name 
                        }
         )

        if custid_int % 2 == 0:
            ret_iddoc=insert_to_doc(custid, 'AoA', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document: IMG_20190512_123820.jpg')  
        elif  custid_int % 3 == 0:
            ret_iddoc=insert_to_doc(custid, 'REG_EXT', 'Y')
            
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document:  IMG_20190512_123820.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124107.jpg',  'Document:  IMG_20190512_124107.jpg')    
            
            ret_iddoc=insert_to_doc(custid, 'COI', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190515_135249.jpg',  'Document:  IMG_20190515_135249.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124206.jpg',  'Document:  IMG_20190512_124206.jpg')    
        elif  custid_int % 5 == 0:
            ret_iddoc=insert_to_doc(custid, 'PoA', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001465.jpg',  '1000001465.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001462.jpg',  '1000001462.jpg') 
        else:    
            ret_iddoc=insert_to_doc(custid, 'REG_EXT', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190511_152919.jpg',  'Document:   IMG_20190511_152919.jpg')            
    connection.commit()
except oracledb.Error as e:
    # Обробка помилок Oracle
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f" Виникла неочікувана помилка: {type(error).__name__}")
    print(f"Деталі помилки: {error}")
    print(sys.exc_info())    
finally:
    # 6. Закриття курсора та з'єднання
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

--- Фізичні особи (Англійська) ---
--- Legal Ent особи (Англійська) ---

Курсор закрито.
З'єднання з БД закрито.


#### Перегляд тестових даних

In [16]:
# SQL- запит як багаторядкова змінна
sql_query = """
    SELECT A.CUSTID, A.CUSTTYPE, A.CUSTNAME, B.TYPEDOC, B.ISACRUAL, F.FILE_NAME, B.IDDOC, F.IDFL
    FROM CUSTDOC.CUST$CUST  A
    LEFT OUTER JOIN CUSTDOC.CUST$DOCS B ON B.CUSTID=A.CUSTID
    LEFT OUTER JOIN CUSTDOC.CUST$DOCS$ATTACH  F ON F.IDDOC=B.IDDOC
 """
# Встановлюємо змінну з'єднання
connection = None
cursor = None

try:
    # 1. Встановлення з'єднання (підключення до бази даних)
    connection = oracledb.connect(dsn=dsn)

    # 2. Створення курсора
    cursor = connection.cursor()
    
    # 3. Виконання запиту SELECT
    cursor.execute(sql_query)
    
    # --- Обробка та відображення даних ---

    
    # Отримання імен стовпців для заголовка
    # cursor.description - це список кортежів, де перший елемент - ім'я стовпця
    column_names = [col[0] for col in cursor.description]
    
    print("-" * 130)
    # Відображення заголовка
    print(f"{column_names[0]:<15} {column_names[1]:<18} {column_names[2]:<20} {column_names[3]:<10} {column_names[4]:<8} {column_names[5]:<38} {column_names[6]:<5} {column_names[7]:<5}")
    print("-" * 130)
    
    # 4. Отримання всіх результатів
    # fetchall() повертає список кортежів, де кожен кортеж - рядок даних
    rows = cursor.fetchall()
  
    # 5. Обробка та відображення отриманих даних
    if rows:
        for row in rows:
            # Виведення даних з форматуванням
            # :<15 означає вирівнювання по лівому краю, займаючи 15 символів
            print(f"{row[0]:<15} {row[1]:<8} {row[2]:<30} {row[3]:<10} {row[4]:<8} {row[5]:<38} {row[6]:<5} {row[7]:<5}")
    else:
        print("Запит не повернув жодного рядка.")
        
    print("-" * 130)
    print(f"Fetched rows={len(rows)}")

except oracledb.Error as e:
    # Обробка помилок Oracle
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    # 6. Закриття курсора та з'єднання
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

----------------------------------------------------------------------------------------------------------------------------------
CUSTID          CUSTTYPE           CUSTNAME             TYPEDOC    ISACRUAL FILE_NAME                              IDDOC IDFL 
----------------------------------------------------------------------------------------------------------------------------------
000000001       CI       Anthony Banks                  NAT_ID     Y        CU_000000001_IMG_20190511_152919.jpg   3615  3616 
000000002       CI       Marcus Walker                  FOR_PASS   Y        CU_000000002_IMG_20190512_123820.jpg   3617  3618 
000000003       CI       William Marsh                  NAT_ID     Y        CU_000000003_IMG_20190512_123820.jpg   3619  3620 
000000003       CI       William Marsh                  NAT_ID     Y        CU_000000003_IMG_20190512_124107.jpg   3619  3621 
000000003       CI       William Marsh                  TAX_ID     Y        CU_000000003_IMG_20190515_1